# 📊 Day 1-00.06: 모델 평가하기 (초보자용)

## 🎯 이번 노트북에서 할 일
- **학습된 모델** 로드하기
- **성능 평가**하기 (매우 간단하게!)
- **원본 모델과 비교**하기
- **결과 분석**하기

## 💡 모델 평가란?
**학습된 모델이 얼마나 잘 작동하는지 확인하는 것**

### 🔧 평가 방법들
1. **정성적 평가**: 사람이 직접 답변 품질 확인
2. **정량적 평가**: 수치로 성능 측정
3. **비교 평가**: 원본 모델과 성능 비교
4. **실제 사용**: 실제 질문에 답변해보기

### 📊 오늘 할 평가
- **질문 답변 품질**: 답변이 정확하고 유용한지
- **RAFT 성능**: 문서를 참고해서 답변하는지
- **한국어 이해**: 한국어 질문을 잘 이해하는지
- **일관성**: 같은 질문에 일관된 답변을 하는지


## 1. 필요한 라이브러리 불러오기


In [ ]:
# 모델 평가에 필요한 라이브러리들을 불러옵니다
import torch
import json
import time
import random
from transformers import (
    AutoTokenizer,           # 토크나이저
    AutoModelForCausalLM,    # 언어 모델
    BitsAndBytesConfig       # 4비트 양자화
)
from peft import (
    PeftModel,               # LoRA 모델 로드
    LoraConfig,              # LoRA 설정
    TaskType                 # 태스크 타입
)
import warnings
warnings.filterwarnings('ignore')

print("✅ 모델 평가 라이브러리가 준비되었습니다!")
print(f"🔥 PyTorch 버전: {torch.__version__}")
print(f"🚀 CUDA 사용 가능: {torch.cuda.is_available()}")


## 2. 학습된 모델 로드하기

> 💡 이 노트북은 개념 학습용 예시입니다. 실제로는 `main-practice/03_fine_tuning_with_lora.ipynb`에서 학습과 저장이 끝났다는 가정하에 결과만 불러옵니다.
> 저장된 모델 파일이 없다면 아래 셀은 샘플 데이터를 사용해 흐름만 보여줍니다.


In [ ]:
# 학습된 모델 로드 과정을 개념적으로 살펴봅니다 (실제 파일 접근 없음)
print("📥 학습된 모델 로드 시나리오")
print("   - 이 노트북은 main-practice/03_fine_tuning_with_lora.ipynb에서 학습과 저장이 끝났다고 가정하고 절차만 설명합니다.")
print("   - Colab 환경에서도 큰 파일을 다시 불러올 필요 없이, 어떤 정보가 저장되었는지만 확인하면 됩니다.")

MODEL_READY = False
model = None
base_model = None
tokenizer = None
training_config = {
    "model_name": "LG/EXAONE-3.0-7.8B-Instruct",
    "lora_config": {
        "r": 16,
        "lora_alpha": 32,
        "target_modules": ["q_proj", "v_proj"],
        "lora_dropout": 0.05,
        "bias": "none"
    },
    "training_args": {
        "num_train_epochs": 3,
        "per_device_train_batch_size": 4,
        "learning_rate": 2e-4
    }
}

print("🗂️ training_config.json 예시")
for key, value in training_config.items():
    print(f"   - {key}: {value}")

print("🔧 LoRA 어댑터 구조 요약 (models/fine_tuned_lora/)")
print("   - adapter_config.json: 위 LoRA 설정이 저장")
print("   - adapter_model.bin: 학습된 LoRA 가중치")
print("   - tokenizer/ : 토크나이저 설정")

print("✅ 실제 프로젝트에서는 위 파일들을 메인 노트북에서 저장한 뒤 여기서 불러와 평가합니다.")
print("📎 MODEL_READY:", MODEL_READY)


## 3. 텍스트 생성 함수 건너뛰기

목업 평가에서는 실제 모델을 호출하지 않습니다. 바로 아래 샘플 데이터(`sample_data`)를 사용해 평가 지표 계산 흐름을 연습합니다.


## 4. 평가 지표 이해하기

학습한 모델을 실제로 불러오는 대신, 예시 답변을 가지고 대표적인 자동 평가 지표를 계산해 봅니다.
- **ROUGE-1**: 정답과 예측 사이의 단어 중복 비율(리콜).
- **BLEU**: 정답 문장을 얼마나 잘 따라 썼는지 보는 정밀도 기반 점수.
- **코사인 유사도**: 두 문장을 벡터로 바꿔 각도를 비교하는 지표(1에 가까울수록 유사).

> 🔁 실제 프로젝트에서는 `generate_text`로 얻은 응답과 정답을 이 지표에 넣어 비교하면 됩니다.


In [ ]:
# 평가용 샘플 데이터 (예시)
import pandas as pd

sample_data = [
    {
        "question": "질문: 인공지능이란 무엇인가요?",
        "reference": "인공지능은 데이터를 학습해 패턴을 찾고 의사결정을 자동화하는 컴퓨터 기술입니다.",
        "baseline_answer": "인공지능은 사람이 만든 규칙을 실행하는 프로그램입니다.",
        "lora_answer": "인공지능은 데이터를 학습해 스스로 패턴을 찾아 의사결정을 돕는 기술입니다."
    },
    {
        "question": "질문: 머신러닝과 딥러닝의 차이점은?",
        "reference": "머신러닝은 사람이 특징을 정의한 뒤 학습시키고, 딥러닝은 신경망이 특징 추출까지 함께 수행합니다.",
        "baseline_answer": "머신러닝은 모델을 학습시키고 딥러닝은 데이터를 수집합니다.",
        "lora_answer": "머신러닝은 사람이 특징을 설계하고, 딥러닝은 신경망이 특징 추출까지 자동으로 수행합니다."
    },
    {
        "question": "질문: 자연어 처리는 무엇인가요?",
        "reference": "자연어 처리는 인간의 언어를 컴퓨터가 이해하고 생성하도록 만드는 기술입니다.",
        "baseline_answer": "자연어 처리는 언어 데이터를 저장하는 기술입니다.",
        "lora_answer": "자연어 처리는 인간의 언어를 컴퓨터가 이해하고 생성하도록 돕는 인공지능 기술입니다."
    }
]

df_samples = pd.DataFrame(sample_data)
print("🗂️ 샘플 데이터 미리보기")
display(df_samples)


In [ ]:
# 평가 지표 계산 함수 정의
import math
import re
from collections import Counter

TOKEN_PATTERN = re.compile(r"[\w가-힣]+", re.UNICODE)

def tokenize(text):
    return TOKEN_PATTERN.findall(text.lower())

def compute_rouge1(reference, candidate):
    ref_tokens = tokenize(reference)
    cand_tokens = tokenize(candidate)
    ref_counts = Counter(ref_tokens)
    overlap = 0
    for token in cand_tokens:
        if ref_counts[token] > 0:
            overlap += 1
            ref_counts[token] -= 1
    return overlap / len(ref_tokens) if ref_tokens else 0.0

def compute_bleu(reference, candidate):
    ref_tokens = tokenize(reference)
    cand_tokens = tokenize(candidate)
    if not cand_tokens:
        return 0.0

    precisions = []
    for n in range(1, 5):
        ref_ngrams = Counter(tuple(ref_tokens[i:i+n]) for i in range(len(ref_tokens) - n + 1))
        cand_ngrams = Counter(tuple(cand_tokens[i:i+n]) for i in range(len(cand_tokens) - n + 1))
        if len(cand_ngrams) == 0:
            precisions.append(0.0)
            continue
        overlap = 0
        for ngram, count in cand_ngrams.items():
            overlap += min(count, ref_ngrams.get(ngram, 0))
        precisions.append(overlap / sum(cand_ngrams.values()))

    if any(p == 0 for p in precisions):
        geo_mean = 0.0
    else:
        geo_mean = math.exp(sum(math.log(p) for p in precisions) / 4)

    ref_len = len(ref_tokens)
    cand_len = len(cand_tokens)
    if cand_len == 0:
        bp = 0.0
    elif cand_len > ref_len:
        bp = 1.0
    else:
        bp = math.exp(1 - ref_len / cand_len)

    return bp * geo_mean

def compute_cosine_similarity(reference, candidate):
    ref_tokens = tokenize(reference)
    cand_tokens = tokenize(candidate)
    vocab = set(ref_tokens) | set(cand_tokens)
    if not vocab:
        return 0.0

    ref_counts = Counter(ref_tokens)
    cand_counts = Counter(cand_tokens)
    dot = sum(ref_counts[token] * cand_counts[token] for token in vocab)
    ref_norm = math.sqrt(sum(count * count for count in ref_counts.values()))
    cand_norm = math.sqrt(sum(count * count for count in cand_counts.values()))
    if ref_norm == 0 or cand_norm == 0:
        return 0.0

    return dot / (ref_norm * cand_norm)

print("🛠️ 평가 지표 함수 정의 완료")


In [ ]:
# 샘플 데이터로 평가 지표 계산
results = []

for row in sample_data:
    reference = row['reference']
    baseline = row['baseline_answer']
    lora = row['lora_answer']

    baseline_scores = {
        'rouge1': compute_rouge1(reference, baseline),
        'bleu': compute_bleu(reference, baseline),
        'cosine': compute_cosine_similarity(reference, baseline)
    }
    lora_scores = {
        'rouge1': compute_rouge1(reference, lora),
        'bleu': compute_bleu(reference, lora),
        'cosine': compute_cosine_similarity(reference, lora)
    }

    results.append({
        '질문': row['question'],
        '기준 모델 ROUGE-1': baseline_scores['rouge1'],
        'LoRA ROUGE-1': lora_scores['rouge1'],
        '기준 모델 BLEU': baseline_scores['bleu'],
        'LoRA BLEU': lora_scores['bleu'],
        '기준 모델 코사인': baseline_scores['cosine'],
        'LoRA 코사인': lora_scores['cosine']
    })

results_df = pd.DataFrame(results)
results_df = results_df.round(3)
print("📈 샘플 평가 결과")
display(results_df)


In [ ]:
# 평균 점수 비교
metric_summary = pd.DataFrame({
    'Metric': ['ROUGE-1', 'BLEU', '코사인 유사도'],
    'Baseline': [
        results_df['기준 모델 ROUGE-1'].mean(),
        results_df['기준 모델 BLEU'].mean(),
        results_df['기준 모델 코사인'].mean()
    ],
    'LoRA': [
        results_df['LoRA ROUGE-1'].mean(),
        results_df['LoRA BLEU'].mean(),
        results_df['LoRA 코사인'].mean()
    ]
}).round(3)

print("📊 평균 성능 비교")
display(metric_summary)

improvements = metric_summary['LoRA'] - metric_summary['Baseline']
for metric, delta in zip(metric_summary['Metric'], improvements):
    arrow = '🔼' if delta > 0 else ('🔽' if delta < 0 else '➖')
    print(f"{arrow} {metric}: {delta:+.3f}")


## 5. 결과 해석하기

- 표를 보면 LoRA 답변이 기준 모델보다 모든 지표에서 높은 값을 갖도록 구성했습니다.
- **ROUGE-1**이 높으면 정답과 단어가 더 많이 겹치므로 내용이 유사하다는 뜻입니다.
- **BLEU**는 n-그램 정밀도를 사용하므로 문장 어순과 표현력이 정답과 비슷할수록 점수가 오릅니다.
- **코사인 유사도**는 문장 벡터의 각도를 비교하므로 전체적인 의미가 비슷할수록 1에 가까워집니다.


### 마무리 정리
- 지금은 예시 데이터를 사용해 ROUGE, BLEU, 코사인 유사도를 계산했습니다.
- 실제 평가에서는 `generate_text`로 얻은 모델 응답을 `sample_data` 대신 넣어 동일한 함수를 호출하면 됩니다.
- 메트릭이 모두 향상되었다면 LoRA 파인튜닝이 효과적이었음을 의미합니다.
- 필요에 따라 METEOR, BERTScore 등 다른 지표도 같은 구조로 확장할 수 있습니다.
